In [ ]:
from typing import Dict
from l5kit.data.map_api import MapAPI
# from l5kit.environment.envs.l5_env import GymStepOutput, SimulationConfigGym
from l5kit.environment.envs.l5_env2 import GymStepOutput, SimulationConfigGym, L5Env2
from l5kit.environment.envs.l5_env import GymStepOutput, SimulationConfigGym, L5Env
# Dataset is assumed to be on the folder specified
# in the L5KIT_DATA_FOLDER environment variable
import gym
import ray
from ray.rllib.agents.ppo import PPOTrainer
from ray.tune.logger import pretty_print
import ray.rllib.algorithms.ppo as ppo
from ray import tune
from ray.rllib.models import ModelCatalog
# from wrapper import L5EnvWrapper
# from src.customEnv.wr import wrapper
# from customModel.customModel import TorchGNCNN, TorchAttentionModel, TorchAttentionModel2
import os

from matplotlib import pyplot as plt
import numpy as np
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from tqdm import tqdm
from tempfile import gettempdir

from l5kit.configs import load_config_data
from l5kit.data import ChunkedDataset, LocalDataManager
from l5kit.dataset import EgoDatasetVectorized
from l5kit.planning.vectorized.closed_loop_model import VectorizedUnrollModel
from l5kit.planning.vectorized.open_loop_model import VectorizedModel
from l5kit.vectorization.vectorizer_builder import build_vectorizer

In [ ]:
# get environment config
from src.constant import SRC_PATH


os.environ["L5KIT_DATA_FOLDER"] = "/workspace/datasets"
env_config_path =  SRC_PATH  + 'src/configs/gym_vectorizer_config.yaml'
# env_config_path = SRC_PATH  + 'src/configs/gym_rasterizer_config.yaml'
# env_config_path = SRC_PATH + 'src/configs/gym_vectorizer_config_hist3.yaml'
dmg = LocalDataManager(None)
cfg = load_config_data(env_config_path)
###############
USE_KIN = False

In [ ]:
# model_path = "/home/pronton/rl/l5kit/examples/urban_driver/OL_HS.pt"
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# model = torch.load(model_path).to(device)
# # model = SAC.load("/home/pronton/rl/l5kit/examples/RL/gg colabs/logs/SAC_640000_steps.zip")
# model = model.eval()
# torch.set_grad_enabled(False)
from src.customModel.customModel import TorchAttentionModel3, TorchAttentionModel4
from src.customModel.customModel import TorchRasterNet


# os.environ["L5KIT_DATA_FOLDER"] = "/media/pronton/linux_files/a100code/l5kit/l5kit_dataset"
# env_config_path = '/home/pronton/rl/rlhf-car/src/configs/gym_vectorizer_config.yaml'
dmg = LocalDataManager(None)
cfg = load_config_data(env_config_path)
# rollout_sim_cfg = SimulationConfigGym()
# rollout_sim_cfg.num_simulation_steps = None
# env_kwargs = {'env_config_path': env_config_path, 'use_kinematic': False, 'sim_cfg': rollout_sim_cfg,  'train': False, 'return_info': True, 'rescale_action': False}
# rollout_env = L5Env2(**env_kwargs)
# print(rollout_env.action_space)
# model = TorchAttentionModel3(np.zeros((112,112,7)), np.array((3,)),3, model_config= {"custom_model_config": {'cfg':cfg}}, name='')
# for param_tensor in model.state_dict():
#     print(param_tensor, "\t", model.state_dict()[param_tensor].size())
###################### TRAINING ######################
ModelCatalog.register_custom_model( "TorchSeparatedAttentionModel", TorchAttentionModel3)
ModelCatalog.register_custom_model( "TorchAttentionModel4", TorchAttentionModel4)
ModelCatalog.register_custom_model( "TorchSeparatedRasterModel", TorchRasterNet)
from src.customEnv.wrapper import L5Env2WrapperTorchCLEReward, L5EnvRasterizerTorch
from ray import tune
import ray
train_eps_length = 32
train_sim_cfg = SimulationConfigGym()
train_sim_cfg.num_simulation_steps = train_eps_length + 1
env_kwargs = {'env_config_path': env_config_path, 'use_kinematic': USE_KIN, 'sim_cfg': train_sim_cfg}
reward_kwargs = {
    'yaw_weight': 1.0,
    'dist_weight': 1.0,
    'cf_weight': 20.0,
    'cr_weight': 20.0,
    'cs_weight': 20.0,
}
tune.register_env("L5-CLE-V2", lambda config: L5Env2WrapperTorchCLEReward(L5Env2(**env_kwargs), reward_kwargs=reward_kwargs))
tune.register_env("L5-CLE-V1", lambda config: L5EnvRasterizerTorch(env = L5Env(**env_kwargs), \
                                                           raster_size= cfg['raster_params']['raster_size'][0], \
                                                           n_channels = 5))
if USE_KIN:
    kin_rescale =  L5Env2(**env_kwargs).kin_rescale
    print('non_kin_rescale:', kin_rescale)
else:
    non_kin_rescale =  L5Env2(**env_kwargs).non_kin_rescale
    print('non_kin_rescale:', non_kin_rescale)


ray.init(num_cpus=4, ignore_reinit_error=True, log_to_driver=False, local_mode=True)
from src.customModel.customKLRewardPPOTrainer import KLPPO
algo = ppo.PPO(
# algo = KLPPO(
        env="L5-CLE-V2", #"L5-CLE-V2",
        config={
            # 'disable_env_checking':True,
            "framework": "torch",
            'log_level': 'INFO',
            'num_gpu': 0,
            'train_batch_size': 1,
            'sgd_minibatch_size': 1,
            'num_sgd_iter': 1,
            'seed': 42,
            'batch_mode': 'truncate_episodes',
            # "rollout_fragment_length": 32,
            "model": {
                "custom_model": "TorchAttentionModel4", #TorchSeparatedAttentionModel
                # Extra kwargs to be passed to your model's c'tor.
                "custom_model_config": {
                    'cfg': cfg,
                    'freeze_for_RLtuning':  True,
                    'load_pretrained': True,
                    'shared_feature_extractor': True,
                    'KL_pretrained': True,
                    # 'future_num_frames':cfg["model_params"]["future_num_frames"],
                    # 'freeze_actor': True,
                    # 'non_kin_rescale': non_kin_rescale,
                    },
            },            
            # "output": "/home/pronton/rl/l5kit/examples/RL/notebooks/logs/l5env2-out", 
            # "output_max_file_size": 5000000,
            '_disable_preprocessor_api': True,
        },
    )

In [ ]:
model_path = "./BPTT.pt"
model_path =  f"{SRC_PATH}src/model/OL_HS.pt"
device = 'cuda:0'
model = torch.load(model_path).to(device)
model = model.eval()
# torch.set_grad_enabled(False)

In [ ]:
from src.customEnv.action_utils import inverseUnicycle, standard_normalizer_nonKin, standard_normalizer_kin

def rollout_episode(model, env, idx = 0, model_type = 'OPENED_LOOP'):
    """Rollout a particular scene index and return the simulation output.

    :param model: the RL policy
    :param env: the gym environment
    :param idx: the scene index to be rolled out
    :return: the episode output of the rolled out scene
    """

    # Set the reset_scene_id to 'idx'
    env.set_reset_id(idx)
    
    # Rollout step-by-step
    obs = env.reset()
    done = False
    rewards = []
    while True:
        # print(obs)
        # action = np.array(env.action_space.sample())
        if model_type == 'CLOSED_LOOP':
            action = model.compute_single_action(obs)
        if model_type == 'OPENED_LOOP':
            if type(obs) == dict:
                obs = {k: torch.as_tensor(v).view(1, *torch.as_tensor(v).shape).to(device) for k, v in obs.items()}
                # print(obs['agent_polyline_availability'])
                # for  k, v in obs.items():
                    # if k == 'agent_polyline_availability':
                    #     obs[k] = torch.as_tensor(v.shape[0] * [True, True, False, False], dtype = bool).view(v.shape[0], 4)
            elif type(obs) == np.ndarray:
                obs = torch.as_tensor(obs).view(1, *obs.shape)
            logits = model(obs)
            if type(logits) == dict: # TODO: Change Vectorized output from dict -> numpy.ndarray
                pred_x = logits['positions'][:,0, 0].view(-1,1)# take the first action 
                pred_y = logits['positions'][:,0, 1].view(-1,1)# take the first action
                pred_yaw = logits['yaws'][:,0,:].view(-1,1)# take the first action
            else:
                batch_size = len(obs)
                predicted = logits.view(batch_size, -1, 3) # B, N, 3 (X,Y,yaw)
                pred_x = predicted[:, 0, 0].view(-1,1) # take the first action 
                pred_y = predicted[:, 0, 1].view(-1,1) # take the first action
                pred_yaw = predicted[:, 0, 2].view(-1,1)# take the first action
            action = torch.cat((pred_x,pred_y, pred_yaw), dim = -1).detach().cpu().numpy()
            # Normalize kin actions
            if USE_KIN:
                action = inverseUnicycle(pred_x, pred_y, pred_yaw, obs['old_speed']) # B, 1
                # print(f'inverse Unicycle: {output_logits}')
                action = standard_normalizer_kin(action).detach().numpy().reshape(-1) # scale actions
                # print(f'normalize actions: {action}')
            else:
                action = standard_normalizer_nonKin(action).reshape(-1)

        # print(action)
        obs, reward, done, info = env.step(action)
        rewards.append(reward)
        # print(action)
        # print(env.ego_output_dict)
        if done:
            break

    # The episode outputs are present in the key "sim_outs"
    sim_out = info["sim_outs"][0]
    return sim_out, np.array(rewards)

In [ ]:
rollout_sim_cfg = SimulationConfigGym()
rollout_sim_cfg.num_simulation_steps = None

env_kwargs = {'env_config_path': env_config_path, 'use_kinematic': USE_KIN, 'sim_cfg': rollout_sim_cfg,  'train': True, 'return_info': True}#, 'rescale_action': True}
# tune.register_env("L5-CLE-V0", lambda config: L5Env(**env_kwargs))
rollout_env = L5EnvRasterizerTorch(env = L5Env(**env_kwargs), \
                                                           raster_size= cfg['raster_params']['raster_size'][0], \
                                                           n_channels = 5)
# rollout_env = L5Env2(**env_kwargs)

In [ ]:
from src.customEnv.wrapper import L5Env2WrapperTorchCLEReward, L5EnvRasterizerTorch
from ray import tune
rollout_sim_cfg = SimulationConfigGym()
rollout_sim_cfg.num_simulation_steps = None
eval_env_kwargs = {'env_config_path': env_config_path, 'use_kinematic': USE_KIN, 'sim_cfg': rollout_sim_cfg, 'train': True, 'return_info': True}
rollout_env = L5Env2WrapperTorchCLEReward(L5Env2(**eval_env_kwargs))

In [ ]:
from copy import deepcopy


model_path =  f"{SRC_PATH}src/model/OL_HS.pt"
model = VectorizedModel(
    history_num_frames_ego=cfg["model_params"]["history_num_frames_ego"],
    history_num_frames_agents=cfg["model_params"]["history_num_frames_agents"],
    num_targets=36, # N (X,Y,Yaw) 36
    weights_scaling=[1,1,1], # 3
    criterion=nn.L1Loss(reduction="none"),
    global_head_dropout=cfg["model_params"]["global_head_dropout"],
    disable_other_agents=cfg["model_params"]["disable_other_agents"],
    disable_map=cfg["model_params"]["disable_map"],
    disable_lane_boundaries=cfg["model_params"]["disable_lane_boundaries"])
pretrained_model = torch.load(model_path)
pretrained_model.eval()
for param_tensor in pretrained_model.state_dict():
    print(param_tensor, "\t", pretrained_model.state_dict()[param_tensor].size())

# model.load_state_dict(torch.load(model_path))
model.load_state_dict(torch.load(model_path).state_dict())
model.eval()

print('-------------')
for param_tensor in model.state_dict():
    print(param_tensor, "\t", model.state_dict()[param_tensor].size())
torch.set_grad_enabled(False)
# for  name, param in model.named_parameters():
#     print(model[name], model[param])

In [ ]:
from l5kit.visualization.visualizer.zarr_utils import episode_out_to_visualizer_scene_gym_cle, simulation_out_to_visualizer_scene
from l5kit.visualization.visualizer.visualizer import visualize
from bokeh.io import output_notebook, show
sim_outs =[]
rollout_sim_cfg = SimulationConfigGym()
rollout_sim_cfg.num_simulation_steps = None
# env_config_path = '/workspace/source/src/configs/gym_config.yaml'

# env_config_path = 
# env_kwargs = {'env_config_path': env_config_path, 'use_kinematic': False, 'sim_cfg': rollout_sim_cfg,  'train': False, 'return_info': True}

# rollout_env = L5Env(**env_kwargs)

# for i in range(1):
# sim_outs.append(rollout_episode(algo, rollout_env, 82, KLPPO))
# results =  np.array([rollout_episode(algo, rollout_env, i, 'CLOSED_LOOP') for i in [7,8,34,13]])
# results = np.array([rollout_episode(algo.get_policy().model.pretrained_policy, rollout_env, i, 'OPENED_LOOP') for i in [70]])
results = np.array([rollout_episode(model, rollout_env, i, 'OPENED_LOOP') for i in [70]])
sim_outs,rewards = results[:,0], np.array(results[:,1])
rewards.reshape(rewards.shape[0], -1)
print([np.sum(r) for r in rewards])

# might change with different rasterizer
mapAPI = MapAPI.from_cfg(dmg, cfg)

def visualize_outputs(sim_outs, map_API):
    for sim_out in sim_outs: # for each scene
        vis_in = episode_out_to_visualizer_scene_gym_cle(sim_out, map_API)
        show(visualize(sim_out.scene_id, vis_in))

output_notebook()
visualize_outputs(sim_outs, mapAPI)

## L5env1

In [ ]:
train_eps_length = 32
train_sim_cfg = SimulationConfigGym()
train_sim_cfg.num_simulation_steps = train_eps_length + 1
env_kwargs = {'env_config_path': env_config_path, 'use_kinematic': False, 'sim_cfg': train_sim_cfg}


tune.register_env("L5-CLE-V1", lambda config: L5EnvWrapper(L5Env(**env_kwargs)))
ray.init(num_cpus=1, ignore_reinit_error=True, log_to_driver=False)
ModelCatalog.register_custom_model(
        "tcnn", TorchGNCNN
    )
# Create the Trainer.
algo = ppo.PPO(
        env="L5-CLE-V1",
        config={
            'num_worker': 1,
            'disable_env_checking':True,
            "framework": "torch",
            # "model": {
            #     "custom_model": "GN_CNN_torch_model",
            #     "custom_model_config": {'feature_dim':128},
            # },
            # "output": "/home/pronton/rl/l5kit/examples/RL/notebooks/logs/l5env2-out", 
            # "output_max_file_size": 5000000,
            '_disable_preprocessor_api': True,
        },
    )

for i in range(2):
    result = algo.train()
    print(pretty_print(result))

In [ ]:

train_eps_length = 10
train_sim_cfg = SimulationConfigGym()
train_sim_cfg.num_simulation_steps = train_eps_length + 1

env_kwargs = {'env_config_path': env_config_path, 'use_kinematic': True, 'sim_cfg': train_sim_cfg}
# env = L5Env2(**env_kwargs)
# env1_kwargs = {'env_config_path': env_config_path, 'use_kinematic': True, 'sim_cfg': train_sim_cfg}
# env1 = L5Env(**env_kwargs)


## L5env2

In [ ]:
algo = ppo.PPO(
        env="L5-CLE-V2",
        config={
            'disable_env_checking':True,
            "framework": "torch",
            'log_level': 'INFO',
            'num_gpu': 1,
            'train_batch_size': 1,
            'sgd_minibatch_size': 1,
            'num_sgd_iter': 1,
            'seed': 42,
            'batch_mode': 'truncate_episodes',
            # "rollout_fragment_length": 32,
            "model": {
                "custom_model": "TorchSharedAttentionModel",
                # Extra kwargs to be passed to your model's c'tor.
                "custom_model_config": {'cfg':cfg},
            },
            # "output": "/home/pronton/rl/l5kit/examples/RL/notebooks/logs/l5env2-out", 
            # "output_max_file_size": 5000000,
            '_disable_preprocessor_api': True,
        },
    )

for i in range(1):
    result = algo.train()
    print(pretty_print(result))